# Abstract
This tutorial aims to be a simple starter kit for people wanting to experiment continual semantic segmentation. To simplify experiments and developments of methods, we propose Continual-Extended-Mnist, a new toy dataset for continual segmentation that reflects the same scenarios explored in recent papers. We explain two of the important challenges, namely catastrophic forgetting coupled with background shift, and show how a naïve baseline systematically fails even on our black-and-white low-resolution images. Then, we introduce a few tricks popularized by the papers “Knowledge Distillation for Incremental Learning in Semantic Segmentation” and “Modeling the Background for Incremental Learning in Semantic Segmentation” (MiB).

# Introduction
If we look at papers on continual learning for classification, most of them experiment on simple datasets such as MNIST, CIFAR10 or Mini-Imagenet. These are small-scale datasets of tiny images which are used to experiment on a relatively simple task: classify object-centered images. On the other hand, the datasets employed to explore the more complex task of continual semantic segmentation are generally Pascal-VOC, ADE20K or COCO: ~10K to 20K high-resolution images with many objects in each image, which requires several hours of training on GPUs, if not days.

While toy datasets are sometimes blamed to be unrealistic as they are much more simple than real-life scenarios, they serve a non negligeable purpose. **They accelerate the research cycle and development of models** as they remove unnecessary complicated variables while keeping most of the essentials. It allows to do many runs of experiments with different models in a fraction of the time to see if it addresses the problems we aim to solve, i.e. in our case catastrophic forgetting especially. If a technique cannot reduce catastrophic forgetting on 60x60 black-and-white images, obviously there is no reason to think that it will solve it on real images of complex scenes.


# Continual Extended Mnist

We owe the generation of the raw data to this [repository](https://github.com/LukeTonin/simple-deep-learning) made by Luke Tonin. He also made an excellent tutorial for [simple semantic segmentation ](https://awaywithideas.com/mnist-extended-a-dataset-for-semantic-segmentation-and-object-detection/) that was an inspiration for ours. We started from this dataset and adapted it to a class-incremental learning scenario, i.e. classes are seen sequentially in successive tasks. In the first task, the model has to learn to segment 0's and 1's, then in the next task it must learn 2's and 3's, then 4's and 5's, and so on. 

Let's first take a look at the dataset.

In [1]:
#from google.colab import drive
#drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import os
import sys
#os.chdir("/content/gdrive/MyDrive/Colab Notebooks/mnist_continual_seg")
sys.path.append("/home/mathieu/Documents/Deep/mnist-continual-seg/simple-deep-learning")
import matplotlib.pyplot as plt
%matplotlib inline  
from scenarios import ContinualMnist

_tasks = {0: [0,1], 1: [2,3], 2: [4,5], 3: [6,7], 4: [8,9]}

2023-06-07 14:49:37.639667: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# Initialize dataloader, optimizer and trainer
from models import simple_seg_model
model = simple_seg_model(n_classes_per_task=[len(_tasks[0])+1])
model = model.cuda()

In [3]:
import torch
from metrics import EvaluaterCallback
from trainer import Trainer_distillation

# Set optimizer
optimizer = torch.optim.Adam(lr = 0.0005, params=model.parameters())
# Set scenario
continual_mnist = ContinualMnist(n_train=5000, n_test=2500, batch_size=72, tasks=_tasks)
# Set Evaluator
evaluater = EvaluaterCallback(model, ["confusion_matrix"], callback_frequency="epoch", n_classes=11, save_matrices=True)
# Set trainer
trainer = Trainer_distillation(model,
                  n_classes=[3],
                  optim=optimizer,
                  from_new_class = 0,
                  lambda_distill=10,
                  output_level_distill=True,
                  callbacks=[evaluater])

In [4]:
from utils import meta_train
meta_train(n_tasks = len(_tasks),
           epochs = 100,
           scenario = continual_mnist, 
           trainer=trainer,
           evaluater=evaluater,
           memory=None)

*******
Task #0
*******
Classes to learn:
-1 0 1
*******


  0%|          | 0/100 [00:00<?, ?it/s]/home/mathieu/.conda/envs/mmdet2.0/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 100/100 [01:52<00:00,  1.13s/it]



Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.992579 |   0.949386 |   0.898333 |
Class IoU
|    |      -1 |        0 |      1 | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 9   |
|---:|--------:|---------:|-------:|:----|:----|:----|:----|:----|:----|:----|:----|
|  0 | 0.99231 | 0.860787 | 0.8419 | X   | X   | X   | X   | X   | X   | X   | X   |
Class Acc
|    |       -1 |        0 |        1 | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 9   |
|---:|---------:|---------:|---------:|:----|:----|:----|:----|:----|:----|:----|:----|
|  0 | 0.995836 | 0.929225 | 0.923098 | X   | X   | X   | X   | X   | X   | X   | X   |

####################################
Next Task
####################################


  0%|          | 0/100 [00:00<?, ?it/s]

*******
Task #1
*******
Classes to learn:
-1 2 3
*******


...


Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |      0.952412 |   0.934121 |   0.684883 |
Class IoU
|    |       -1 |        0 |        1 |       2 |        3 |        4 |        5 |        6 |        7 | 8   | 9   |
|---:|---------:|---------:|---------:|--------:|---------:|---------:|---------:|---------:|---------:|:----|:----|
|  0 | 0.953539 | 0.575764 | 0.407726 | 0.62219 | 0.639803 | 0.755769 | 0.723201 | 0.779511 | 0.706445 | X   | X   |
Class Acc
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |        6 |        7 | 8   | 9   |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|:----|:----|
|  0 | 0.954455 | 0.985149 | 0.963055 | 0.937669 | 0.945732 | 0.929352 | 0.897496 | 0.911292 | 0.882892 | X   | X   |

####################################
Next Task
####################################


  0%|          | 0/100 [00:00<?, ?it/s]

*******
Task #4
*******
Classes to learn:
-1 8 9
*******


100%|██████████| 100/100 [03:34<00:00,  2.14s/it]


Overall stats
|    |   Overall Acc |   Mean Acc |   Mean IoU |
|---:|--------------:|-----------:|-----------:|
|  0 |       0.93868 |   0.913134 |   0.672425 |
Class IoU
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |        6 |        7 |        8 |        9 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|
|  0 | 0.941884 | 0.562879 | 0.284181 | 0.619574 | 0.602511 | 0.711016 | 0.685136 | 0.807329 | 0.729055 | 0.741394 | 0.711721 |
Class Acc
|    |       -1 |        0 |        1 |        2 |        3 |        4 |        5 |        6 |        7 |        8 |        9 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|
|  0 | 0.942684 | 0.975792 | 0.962253 | 0.913266 | 0.945256 | 0.905136 | 0.864729 | 0.904977 | 0.890334 | 0.870879 | 0.869166 |


In [8]:
evaluater.create_animation('experiments/2-2/1000/72/100/knowledge-distillation_task', sample_freq=99)

TypeError: 'float' object cannot be interpreted as an integer